<a href="https://colab.research.google.com/github/karriechey/DeyerleNotebook/blob/main/WiDS_Team7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load training datasets
train_solutions = pd.read_excel('/content/drive/MyDrive/WiDs Team7/data/train/TRAINING_SOLUTIONS.xlsx')
train_categorical = pd.read_excel('/content/drive/MyDrive/WiDs Team7/data/train/TRAIN_CATEGORICAL_METADATA.xlsx')
train_quantitative = pd.read_excel('/content/drive/MyDrive/WiDs Team7/data/train/TRAIN_QUANTITATIVE_METADATA.xlsx')
train_fmri = pd.read_csv('/content/drive/MyDrive/WiDs Team7/data/train/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv')

# Load test datasets
test_categorical = pd.read_excel('/content/drive/MyDrive/WiDs Team7/data/test//TEST_CATEGORICAL.xlsx')
test_quantitative = pd.read_excel('/content/drive/MyDrive/WiDs Team7/data/test/TEST_QUANTITATIVE_METADATA.xlsx')
test_fmri = pd.read_csv('/content/drive/MyDrive/WiDs Team7/data/test/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')

# Inspect the first few rows and get data types
print(train_solutions.head())
print(train_solutions.info())

# Drop rows with missing values in any column
train_solutions = train_solutions.dropna()

# Fill missing values with the median, but only for numeric columns
train_solutions_numeric = train_solutions.select_dtypes(include=['float64', 'int64'])
train_solutions[train_solutions_numeric.columns] = train_solutions_numeric.fillna(train_solutions_numeric.median())

# One-hot encode categorical columns
train_solutions = pd.get_dummies(train_solutions, columns=['column_name'])

# Standardize numerical columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_solutions['numerical_column'] = scaler.fit_transform(train_solutions[['numerical_column']])


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/WiDs Team7/data/train/TRAINING_SOLUTIONS.xlsx'

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Check for missing values
print("Missing values in training data:")
print(train_solutions.isnull().sum())
print(train_categorical.isnull().sum())
print(train_quantitative.isnull().sum())
print(train_fmri.isnull().sum())

# Handle missing values
train_solutions.fillna(train_solutions.mean(), inplace=True)
train_categorical.fillna(train_categorical.mode().iloc[0], inplace=True)
train_quantitative.fillna(train_quantitative.mean(), inplace=True)
train_fmri.fillna(train_fmri.mean(), inplace=True)

# Separate the numeric columns from the non-numeric ones
numeric_columns = train_solutions.select_dtypes(include=['number']).columns

# Fill missing values for numeric columns with the mean of each column
train_solutions[numeric_columns] = train_solutions[numeric_columns].fillna(train_solutions[numeric_columns].mean())

# Fill non-numeric columns (e.g., categorical) with a placeholder 'Unknown'
non_numeric_columns = train_solutions.select_dtypes(exclude=['number']).columns
train_solutions[non_numeric_columns] = train_solutions[non_numeric_columns].fillna('Unknown')

# One-hot encoding for categorical data
categorical_cols = train_categorical.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse=False, drop='first')  # Avoid multicollinearity by dropping the first column
train_categorical_encoded = encoder.fit_transform(train_categorical[categorical_cols])

# Scaling quantitative data
scaler = StandardScaler()
train_quantitative_scaled = scaler.fit_transform(train_quantitative)

# Combine processed data into a final DataFrame for training
train_data = pd.concat([train_solutions, pd.DataFrame(train_categorical_encoded), pd.DataFrame(train_quantitative_scaled)], axis=1)

# Display shapes and head of the data
print(train_data.shape)
print(train_data.head())

Missing values in training data:


NameError: name 'train_solutions' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split data into features (X) and target (y)
X = train.drop('target_column', axis=1)
y = train['target_column']

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on validation set
y_pred = model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy}")

NameError: name 'train' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score


# Load training datasets
train_solutions = pd.read_excel('/content/drive/MyDrive/WiDs Team7/data/train/TRAINING_SOLUTIONS.xlsx')
train_categorical = pd.read_excel('/content/drive/MyDrive/WiDs Team7/data/train/TRAIN_CATEGORICAL_METADATA.xlsx')
train_quantitative = pd.read_excel('/content/drive/MyDrive/WiDs Team7/data/train/TRAIN_QUANTITATIVE_METADATA.xlsx')
train_fmri = pd.read_csv('/content/drive/MyDrive/WiDs Team7/data/train/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv')

# Load test datasets
test_categorical = pd.read_excel('/content/drive/MyDrive/WiDs Team7/data/test//TEST_CATEGORICAL.xlsx')
test_quantitative = pd.read_excel('/content/drive/MyDrive/WiDs Team7/data/test/TEST_QUANTITATIVE_METADATA.xlsx')
test_fmri = pd.read_csv('/content/drive/MyDrive/WiDs Team7/data/test/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')

# Ensure participant_id is unique in all datasets
train_solutions = train_solutions.drop_duplicates(subset=['participant_id'])
train_categorical = train_categorical.drop_duplicates(subset=['participant_id'])
train_quantitative = train_quantitative.drop_duplicates(subset=['participant_id'])
train_fmri = train_fmri.drop_duplicates(subset=['participant_id'])

# Merge data with targets
train_data = train_categorical.merge(train_quantitative, on='participant_id', how='inner')
train_data = train_data.merge(train_fmri, on='participant_id', how='inner')
train_data = train_data.merge(train_solutions, on='participant_id', how='inner')

# Preprocessing: Handle missing values in numeric columns only
numeric_cols = train_data.select_dtypes(include=['float64', 'int64']).columns
train_data[numeric_cols] = train_data[numeric_cols].fillna(train_data[numeric_cols].median())

# One-hot encode categorical variables (excluding participant_id)
categorical_cols = train_categorical.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
encoded_categorical = encoder.fit_transform(train_categorical[categorical_cols])
encoded_feature_names = encoder.get_feature_names_out(categorical_cols)

# Standardize numerical features
scaler = StandardScaler()
quantitative_scaled = scaler.fit_transform(train_quantitative.drop(columns=['participant_id']))

# Combine processed data
feature_names = list(encoded_feature_names) + list(train_quantitative.drop(columns=['participant_id']).columns) + list(train_fmri.drop(columns=['participant_id']).columns)
X = np.hstack([encoded_categorical, quantitative_scaled, train_fmri.drop(columns=['participant_id']).values])
y_adhd = train_data['ADHD_Outcome']
y_sex = train_data['Sex_F']

# Split data
X_train, X_val, y_train_adhd, y_val_adhd = train_test_split(X, y_adhd, test_size=0.2, random_state=42)
X_train, X_val, y_train_sex, y_val_sex = train_test_split(X, y_sex, test_size=0.2, random_state=42)

# Model training (XGBoost for better performance)
xgb_adhd = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_sex = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

xgb_adhd.fit(X_train, y_train_adhd)
xgb_sex.fit(X_train, y_train_sex)

# Predictions on validation set
y_pred_adhd = xgb_adhd.predict(X_val)
y_pred_sex = xgb_sex.predict(X_val)

# Evaluate using weighted F1-score
f1_adhd = f1_score(y_val_adhd, y_pred_adhd)
f1_sex = f1_score(y_val_sex, y_pred_sex)
average_f1 = (f1_adhd + f1_sex) / 2

print(f"F1 Score ADHD: {f1_adhd:.4f}")
print(f"F1 Score Sex: {f1_sex:.4f}")
print(f"Final Average F1 Score: {average_f1:.4f}")

# Now process the test data and generate predictions
print("\nGenerating predictions for test data...")

# Prepare test data in the same way as training data
test_data = test_categorical.merge(test_quantitative, on='participant_id', how='inner')
test_data = test_data.merge(test_fmri, on='participant_id', how='inner')

# Handle missing values in numeric columns
numeric_cols = test_data.select_dtypes(include=['float64', 'int64']).columns
test_data[numeric_cols] = test_data[numeric_cols].fillna(test_data[numeric_cols].median())

# One-hot encode categorical variables using the same encoder
categorical_cols = test_categorical.select_dtypes(include=['object']).columns
encoded_test_categorical = encoder.transform(test_categorical[categorical_cols])

# Scale numerical features using the same scaler
test_quantitative_scaled = scaler.transform(test_quantitative.drop(columns=['participant_id']))

# Combine processed test data
X_test = np.hstack([
    encoded_test_categorical,
    test_quantitative_scaled,
    test_fmri.drop(columns=['participant_id']).values
])

# Get participant IDs for the final submission
participant_ids = test_data['participant_id'].values

# Make predictions
test_pred_adhd = xgb_adhd.predict(X_test)
test_pred_sex = xgb_sex.predict(X_test)

# Create submission DataFrame
submission = pd.DataFrame({
    'participant_id': participant_ids,
    'ADHD_Outcome': test_pred_adhd.astype(int),
    'Sex_F': test_pred_sex.astype(int)
})

# Save to CSV in the required format
submission.to_csv('submission.csv', index=False)

# To match the exact format shown in the sample
with open('submission_exact_format.csv', 'w') as f:
    f.write('participant_id\n\nADHD_Outcome\nSex_F\n')
    for _, row in submission.iterrows():
        f.write(f"{row['participant_id']}\n{int(row['ADHD_Outcome'])}\n{int(row['Sex_F'])}\n")

print("Submission files created!")
print("Preview of submission data:")
print(submission.head())

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:56:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:57:56] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:59:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
